# Load List of Kings and Queens
Load from xlsx because:
* CSVs are corrupt
* API from Parlamentsdienste is a pain
Download at the bottom: https://www.zh.ch/de/politik-staat/wahlen-abstimmungen/kantons-regierungsratswahlen/mitglieder-kantonsrats-ab-1803.html#-373901206

In [1]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import numpy as np

In [2]:
df_person = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Personen')
df_funktion = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Funktionen')

# Take newest Party. Could be improved
df_partei = pd.read_excel(Path('../data/kantonsrathistory.xlsx'), 'Parteien')
df_partei = df_partei.sort_values('DATUM_VON_JAHR', ascending=False)
df_partei = df_partei.drop_duplicates(['ID_PERSON_NEW'], keep='first')

In [3]:
df = df_funktion.merge(df_person, on = 'ID_PERSON_NEW')
df = df.merge(df_partei[['ID_PERSON_NEW', 'PARTEIBEZEICHNUNG']], on = 'ID_PERSON_NEW')

df = df[(df.DATUM_VON_JAHR >= 1980) | (df.DATUM_VON_JAHR.isna())]


records = []
for i, row in df.iterrows():

    try:
        if not np.isnan(row['DATUM_VON_JAHR']) and not(np.isnan(row['DATUM_VON_MONAT'])):
            start = datetime(int(row['DATUM_VON_JAHR']), int(row['DATUM_VON_MONAT']), int(row['DATUM_VON_TAG']))
        else:
            start = datetime(1700, 1, 1)

        if (not np.isnan(row['DATUM_BIS_JAHR'])) and (not np.isnan(row['DATUM_BIS_MONAT'])):
            end = datetime(int(row['DATUM_BIS_JAHR']), int(row['DATUM_BIS_MONAT']), int(row['DATUM_BIS_TAG']))
        else:
            end = datetime(9999, 12, 31, 23, 59, 59)
    except Exception as e:
        print(e)
        print(row)
        raise

    record = {
        'id': row['ID_PERSON_NEW'],
        'name': row['NACHNAME'],
        'vorname': row['VORNAME'],
        'geschlecht': row['GESCHLECHT'],
        'jahrgang': row['DATUM_GEBURT_JAHR'],
        'partei': row['PARTEIBEZEICHNUNG'],
        'funktion': row['FUNKTIONSBEZEICHNUNG'],
        'start': datetime.strftime(start, '%Y-%m-%dT00:00:00.000'),
        'end': datetime.strftime(end, '%Y-%m-%dT00:00:00.000'),
    }
    records.append(record)

df_res = pd.DataFrame(records)
df_res.to_csv(Path('../export/mitglieder.csv'), index=False)

print("finito")

finito


In [4]:
df_res.head()

,id,name,vorname,geschlecht,jahrgang,partei,funktion,start,end
0,13,Petermann,Albert L.,m,1937.0,FDP,NaN,1700-01-01T00:00:00.000,9999-12-31T00:00:00.000
1,39,Ruedin,Jacques,m,1910.0,Ch,NaN,1700-01-01T00:00:00.000,9999-12-31T00:00:00.000
2,45,Nievergelt,Ernst,m,1894.0,F,NaN,1700-01-01T00:00:00.000,9999-12-31T00:00:00.000
3,46,Reimann,Hansheinrich,m,1914.0,F,NaN,1700-01-01T00:00:00.000,9999-12-31T00:00:00.000
4,52,Fischer,Otto,m,1901.0,PdA,NaN,1700-01-01T00:00:00.000,9999-12-31T00:00:00.000
